In [1]:
import cv2

In [2]:
import imutils

In [3]:
import pytesseract

In [4]:
pytesseract.pytesseract.tesseract_cmd=r"C:/Users/hp/AppData/Local/Tesseract-OCR/tesseract.exe"

In [5]:
image=cv2.imread("bike.jpg")

In [6]:
image=imutils.resize(image,width=500)

In [7]:
cv2.imshow("original image",image)
cv2.waitKey(0)

-1

In [8]:
gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
cv2.imshow("Gray Scale Image",gray)
cv2.waitKey(0)

-1

In [9]:
gray=cv2.bilateralFilter(gray,11,17,17)
cv2.imshow("Smoother Image",gray)
cv2.waitKey(0)


-1

In [10]:
edged=cv2.Canny(gray,170,200)
cv2.imshow("Canny Edge",edged)
cv2.waitKey(0)

-1

In [11]:
cnts,new=cv2.findContours(edged.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
image1=image.copy()
cv2.drawContours(image1,cnts,-1,(0,255,0),3)
cv2.imshow("Canny After Contouring",image1)
cv2.waitKey(0)

-1

In [12]:
cnts=sorted(cnts,key=cv2.contourArea,reverse=True)[:30]
NumberPlateCount=None

In [13]:
image2=image.copy()
cv2.drawContours(image2,cnts,-1,(0,255,0),3)
cv2.imshow("Top 30 Contours",image2)
cv2.waitKey(0)

-1

In [14]:
count=0
name=1
for i in cnts:
    perimeter=cv2.arcLength(i,True)
    approx=cv2.approxPolyDP(i,0.02*perimeter,True)
    if(len(approx)==4):
        NumberPlateCount=approx
        x,y,w,h=cv2.boundingRect(i)
        crp_img=image[y:y+h,x:x+w]
        cv2.imwrite(str(name)+'.png',crp_img)
        name+=1
        break
cv2.drawContours(image,[NumberPlateCount],-1,(0,255,0),3)
cv2.imshow("Final Image",image)
cv2.waitKey(0)

-1

In [15]:
crp_img_loc='1.png'
cv2.imshow("Cropped Image",cv2.imread(crp_img_loc))


In [16]:
text=pytesseract.image_to_string(crp_img_loc)
print("Number is: ",text)
cv2.waitKey(0)

Number is:  T_upsoH' 3144


-1

In [17]:
#oracle connection
import cx_Oracle

In [18]:
try:
    conn = cx_Oracle.connect("system/Cdac1234@localhost:1521/orcl")
except Exception as err:
    print("Error while creating the connection",err)
else:
    print(conn.version)
    try:
        cur=conn.cursor()
        #sql_insert="""INSERT INTO p VALUES(:1)""",(text)
        
        cur.execute("INSERT INTO p VALUES(:1)",(text,))
    except Exception as err:
        print("error while inserting the data",err)
    else:
        print("insert completed")
        conn.commit()
finally:
    cur.close()
    conn.close()

11.2.0.1.0
insert completed


In [19]:
#pdf generation
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import Table, TableStyle, Paragraph
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.styles import getSampleStyleSheet
import cx_Oracle

try:
    conn = cx_Oracle.connect("system/Cdac1234@localhost:1521/orcl")
except Exception as err:
    print("Error while creating the connection",err)
else:
    print(conn.version)
    try:
        cur=conn.cursor()
        sql_extract="""SELECT * from p WHERE rownum > ( SELECT (MAX(ROWNUM)-10) FROM p)"""
        x=cur.execute(sql_extract)
        for r in x:
            y=str(r)
            print(y[1:-2])
            
    except Exception as err:
        print("error while inserting the data",err)
    else:
        print("insert completed")
        conn.commit()
finally:
    cur.close()
    conn.close()
    
    
    


11.2.0.1.0
'urm 131429455’'
'urm 131429455’'
'urm 131429455’'
'urm 131429455’'
"T_upsoH' 3144"
insert completed


In [20]:
styles = getSampleStyleSheet()
style = styles["BodyText"]

canv = Canvas("doc105.pdf", pagesize=letter)

header = Paragraph("<bold><font size=18>e-CHALLAN</font></bold>", style)

data = [['number_plate', 'challan_amount'],
        [y[1:-2], '500']]
t = Table(data)
t.setStyle(TableStyle([("BOX", (0, 0), (-1, -1), 0.25, colors.black),
                       ('INNERGRID', (0, 0), (-1, -1), 0.25, colors.black)]))
data_len = len(data)

for each in range(data_len):
    if each % 2 == 0:
        bg_color = colors.whitesmoke
    else:
        bg_color = colors.lightgrey

    t.setStyle(TableStyle([('BACKGROUND', (0, each), (-1, each), bg_color)]))

aW = 540
aH = 720

w, h = header.wrap(aW, aH)
header.drawOn(canv, 72, aH)
aH = aH - h
w, h = t.wrap(aW, aH)
t.drawOn(canv, 72, aH-h)
canv.save()